# 109 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

2023-10-01 14:08:11.444310: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 14:08:11.495740: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-01 14:08:11.495786: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-01 14:08:11.495816: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-01 14:08:11.504292: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 14:08:11.505315: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [3]:
no_of_patients=109

In [4]:
train=sorted(glob('files/*.edf'))
train=train[:no_of_patients]
train

['files/S001R03.edf',
 'files/S002R03.edf',
 'files/S003R03.edf',
 'files/S004R03.edf',
 'files/S005R03.edf',
 'files/S006R03.edf',
 'files/S007R03.edf',
 'files/S008R03.edf',
 'files/S009R03.edf',
 'files/S010R03.edf',
 'files/S011R03.edf',
 'files/S012R03.edf',
 'files/S013R03.edf',
 'files/S014R03.edf',
 'files/S015R03.edf',
 'files/S016R03.edf',
 'files/S017R03.edf',
 'files/S018R03.edf',
 'files/S019R03.edf',
 'files/S020R03.edf',
 'files/S021R03.edf',
 'files/S022R03.edf',
 'files/S023R03.edf',
 'files/S024R03.edf',
 'files/S025R03.edf',
 'files/S026R03.edf',
 'files/S027R03.edf',
 'files/S028R03.edf',
 'files/S029R03.edf',
 'files/S030R03.edf',
 'files/S031R03.edf',
 'files/S032R03.edf',
 'files/S033R03.edf',
 'files/S034R03.edf',
 'files/S035R03.edf',
 'files/S036R03.edf',
 'files/S037R03.edf',
 'files/S038R03.edf',
 'files/S039R03.edf',
 'files/S040R03.edf',
 'files/S041R03.edf',
 'files/S042R03.edf',
 'files/S043R03.edf',
 'files/S044R03.edf',
 'files/S045R03.edf',
 'files/S0

In [5]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [8]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(1308000, 163391, 1308000, 163391)

In [10]:
print(xtest.iloc[:,-1].values)

[-5.7e-05 -4.5e-05 -2.9e-05 ...  8.0e-06 -7.0e-06 -5.0e-06]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000133,0.000002,0.000150,-0.000019,-0.000049,-0.000018,-0.000034,0.000027,0.000001,-0.000059,...,0.000002,0.000072,0.000035,-0.000071,-0.000008,0.000098,0.000042,-0.000033,0.000012,0.000007
1307996,0.000133,-0.000022,0.000124,-0.000035,-0.000058,-0.000049,-0.000029,0.000025,-0.000013,-0.000086,...,-0.000014,0.000077,0.000039,-0.000070,-0.000012,0.000078,0.000053,-0.000025,0.000008,0.000005
1307997,0.000134,-0.000031,0.000087,-0.000032,-0.000054,0.000003,-0.000025,0.000028,-0.000020,-0.000081,...,-0.000024,0.000093,0.000052,-0.000067,-0.000015,0.000071,0.000063,-0.000025,0.000001,0.000007
1307998,0.000134,-0.000022,0.000102,-0.000029,-0.000045,-0.000006,-0.000018,0.000039,-0.000010,-0.000080,...,-0.000014,0.000100,0.000059,-0.000058,-0.000010,0.000083,0.000060,-0.000030,0.000001,0.000002


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_46024/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_46024/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_46024/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000133,0.000002,0.000150,-0.000019,-0.000049,-0.000018,-0.000034,0.000027,0.000001,-0.000059,...,0.000002,0.000072,0.000035,-0.000071,-0.000008,0.000098,0.000042,-0.000033,0.000012,0.000007
1307996,0.000133,-0.000022,0.000124,-0.000035,-0.000058,-0.000049,-0.000029,0.000025,-0.000013,-0.000086,...,-0.000014,0.000077,0.000039,-0.000070,-0.000012,0.000078,0.000053,-0.000025,0.000008,0.000005
1307997,0.000134,-0.000031,0.000087,-0.000032,-0.000054,0.000003,-0.000025,0.000028,-0.000020,-0.000081,...,-0.000024,0.000093,0.000052,-0.000067,-0.000015,0.000071,0.000063,-0.000025,0.000001,0.000007
1307998,0.000134,-0.000022,0.000102,-0.000029,-0.000045,-0.000006,-0.000018,0.000039,-0.000010,-0.000080,...,-0.000014,0.000100,0.000059,-0.000058,-0.000010,0.000083,0.000060,-0.000030,0.000001,0.000002


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10


40875/40875 [==============================] - 58s 1ms/step - loss: 3.0354 - val_loss: 3.1724
Epoch 2/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.5684 - val_loss: 3.0737
Epoch 3/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.4306 - val_loss: 3.1743
Epoch 4/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.3731 - val_loss: 3.1118
Epoch 5/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.3384 - val_loss: 3.1032
Epoch 6/10
40875/40875 [==============================] - 56s 1ms/step - loss: 2.3152 - val_loss: 3.1299
Epoch 7/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.2978 - val_loss: 3.1464
Epoch 8/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.2854 - val_loss: 3.1053
Epoch 9/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.2739 - val_loss: 3.1682
Epoch 10/10
40875/40875 [==============================] - 56s 1ms

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

  59/5106 [..............................] - ETA: 4s  

5106/5106 [==============================] - 4s 842us/step
              precision    recall  f1-score   support

           0       0.10      0.10      0.10      1499
           1       0.14      0.13      0.14      1499
           2       0.19      0.28      0.22      1499
           3       0.42      0.46      0.44      1499
           4       0.52      0.65      0.58      1499
           5       0.37      0.52      0.43      1499
           6       0.34      0.54      0.42      1499
           7       0.54      0.47      0.50      1499
           8       0.51      0.49      0.50      1499
           9       0.17      0.25      0.20      1499
          10       0.18      0.30      0.23      1499
          11       0.14      0.17      0.15      1499
          12       0.22      0.12      0.16      1499
          13       0.12      0.10      0.11      1499
          14       0.10      0.01      0.02      1499
          15       0.44      0.28      0.34      1499
          16       0.0

## 0-16

In [19]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [20]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_46024/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_46024/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_46024/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000133,0.000002,0.000150,-0.000019,-0.000049,-0.000018,-0.000034,0.000027,0.000001,-0.000059,...,0.000002,0.000072,0.000035,-0.000071,-0.000008,0.000098,0.000042,-0.000033,0.000012,0.000007
1307996,0.000133,-0.000022,0.000124,-0.000035,-0.000058,-0.000049,-0.000029,0.000025,-0.000013,-0.000086,...,-0.000014,0.000077,0.000039,-0.000070,-0.000012,0.000078,0.000053,-0.000025,0.000008,0.000005
1307997,0.000134,-0.000031,0.000087,-0.000032,-0.000054,0.000003,-0.000025,0.000028,-0.000020,-0.000081,...,-0.000024,0.000093,0.000052,-0.000067,-0.000015,0.000071,0.000063,-0.000025,0.000001,0.000007
1307998,0.000134,-0.000022,0.000102,-0.000029,-0.000045,-0.000006,-0.000018,0.000039,-0.000010,-0.000080,...,-0.000014,0.000100,0.000059,-0.000058,-0.000010,0.000083,0.000060,-0.000030,0.000001,0.000002


In [21]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
40875/40875 [==============================] - 56s 1ms/step - loss: 2.2505 - val_loss: 2.6419
Epoch 2/10
40875/40875 [==============================] - 55s 1ms/step - loss: 1.6046 - val_loss: 2.7227
Epoch 3/10
40875/40875 [==============================] - 55s 1ms/step - loss: 1.4743 - val_loss: 2.7302
Epoch 4/10
40875/40875 [==============================] - 57s 1ms/step - loss: 1.3828 - val_loss: 2.8202
Epoch 5/10
40875/40875 [==============================] - 56s 1ms/step - loss: 1.3165 - val_loss: 2.7990
Epoch 6/10
40875/40875 [==============================] - 56s 1ms/step - loss: 1.2763 - val_loss: 2.7809
Epoch 7/10
40875/40875 [==============================] - 56s 1ms/step - loss: 1.2484 - val_loss: 2.7680
Epoch 8/10
40875/40875 [==============================] - 56s 1ms/step - loss: 1.2260 - val_loss: 2.8155
Epoch 9/10
40875/40875 [==============================] - 56s 1ms/step - loss: 1.2099 - val_loss: 2.9393
Epoch 10/10
40875/40875 [==============================

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

  54/5106 [..............................] - ETA: 4s  

5106/5106 [==============================] - 4s 836us/step
              precision    recall  f1-score   support

           0       0.34      0.22      0.27      1499
           1       0.26      0.37      0.31      1499
           2       0.25      0.17      0.20      1499
           3       0.66      0.32      0.43      1499
           4       0.78      0.75      0.76      1499
           5       0.54      0.59      0.56      1499
           6       0.63      0.71      0.67      1499
           7       0.82      0.56      0.67      1499
           8       0.52      0.61      0.56      1499
           9       0.37      0.41      0.38      1499
          10       0.44      0.55      0.49      1499
          11       0.42      0.51      0.46      1499
          12       0.32      0.24      0.28      1499
          13       0.26      0.25      0.25      1499
          14       0.21      0.08      0.11      1499
          15       0.43      0.09      0.15      1499
          16       0.2

## 0-32

In [25]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [26]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_46024/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_46024/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_46024/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [27]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
40875/40875 [==============================] - 57s 1ms/step - loss: 1.6743 - val_loss: 2.3206
Epoch 2/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.8609 - val_loss: 2.2545
Epoch 3/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.6955 - val_loss: 2.0811
Epoch 4/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.6108 - val_loss: 2.1282
Epoch 5/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.5554 - val_loss: 2.0657
Epoch 6/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.5165 - val_loss: 2.2313
Epoch 7/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.4892 - val_loss: 2.2417
Epoch 8/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.4677 - val_loss: 2.1978
Epoch 9/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.4499 - val_loss: 2.2238
Epoch 10/10
40875/40875 [==============================

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

5106/5106 [==============================] - 4s 836us/step
              precision    recall  f1-score   support

           0       0.69      0.77      0.73      1499
           1       0.55      0.78      0.64      1499
           2       0.57      0.70      0.63      1499
           3       0.61      0.35      0.45      1499
           4       0.87      0.90      0.88      1499
           5       0.85      0.95      0.90      1499
           6       0.96      0.91      0.93      1499
           7       0.80      0.77      0.79      1499
           8       0.41      0.61      0.49      1499
           9       0.54      0.73      0.62      1499
          10       0.91      0.88      0.89      1499
          11       0.91      0.93      0.92      1499
          12       0.79      0.61      0.69      1499
          13       0.67      0.91      0.77      1499
          14       0.40      0.65      0.50      1499
          15       0.87      0.29      0.44      1499
          16       0.2

## 0-64

In [31]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [32]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
34003/40875 [=======================>......] - ETA: 8s - loss: 1.3329

40875/40875 [==============================] - 59s 1ms/step - loss: 1.2285 - val_loss: 2.3309
Epoch 2/10
40875/40875 [==============================] - 58s 1ms/step - loss: 0.5334 - val_loss: 2.1729
Epoch 3/10
40875/40875 [==============================] - 58s 1ms/step - loss: 0.3822 - val_loss: 2.0420
Epoch 4/10
40875/40875 [==============================] - 58s 1ms/step - loss: 0.3033 - val_loss: 1.9265
Epoch 5/10
40875/40875 [==============================] - 58s 1ms/step - loss: 0.2580 - val_loss: 1.6510
Epoch 6/10
40875/40875 [==============================] - 58s 1ms/step - loss: 0.2266 - val_loss: 1.8676
Epoch 7/10
40875/40875 [==============================] - 58s 1ms/step - loss: 0.2038 - val_loss: 1.7308
Epoch 8/10
40875/40875 [==============================] - 58s 1ms/step - loss: 0.1851 - val_loss: 1.6792
Epoch 9/10
40875/40875 [==============================] - 58s 1ms/step - loss: 0.1708 - val_loss: 1.6478
Epoch 10/10
40875/40875 [==============================] - 58s 1ms

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

  60/5106 [..............................] - ETA: 4s  

5106/5106 [==============================] - 5s 919us/step
              precision    recall  f1-score   support

           0       0.75      0.95      0.84      1499
           1       0.89      0.89      0.89      1499
           2       0.65      0.63      0.64      1499
           3       0.93      0.97      0.95      1499
           4       0.98      1.00      0.99      1499
           5       0.95      0.97      0.96      1499
           6       0.95      0.95      0.95      1499
           7       0.97      0.60      0.74      1499
           8       0.60      0.60      0.60      1499
           9       0.47      0.84      0.60      1499
          10       0.83      0.91      0.86      1499
          11       0.96      0.91      0.93      1499
          12       0.80      0.76      0.78      1499
          13       0.83      0.81      0.82      1499
          14       0.76      0.68      0.72      1499
          15       0.98      0.41      0.58      1499
          16       0.6